## Offline Open-Source Chatbot

In [1]:
# =========================
# Physics-Teacher Chatbot (Ollama edition)
# =========================

import json
import requests
import gradio as gr

# -------------------------
# Ollama connection settings
# -------------------------
OLLAMA_URL = "http://localhost:11434/api/chat"   # default daemon address
AVAILABLE_MODELS = ["tinyllama", "phi3:3.8b","llama3:8b", "qwen2.5-coder:7b"]

# -------------------------
# System prompt (same as before)
# -------------------------
system_message = ("You are a helpful general-purpose coding assistant")

In [2]:
# -------------------------
# Core streaming chat function
# -------------------------
def stream_chat(message: str, history: list, model_choice: str):
    """
    Generates streaming responses from the selected Ollama model
    while preserving the full chat history (memory).
    """
    # 1) Build chat history in Ollama format
    messages = [{"role": "system", "content": system_message}]
    for user_msg, bot_msg in history:
        messages.append({"role": "user", "content": user_msg})
        messages.append({"role": "assistant", "content": bot_msg})
    messages.append({"role": "user", "content": message})

    # 2) Make a streaming request to Ollama
    response = requests.post(
        OLLAMA_URL,
        json={
            "model": model_choice,
            "messages": messages,
            "stream": True,       # Enable server-sent-events streaming
            "options": {          # ← new
                "num_thread": 2   # choose 1‒N  (0 = let Ollama decide)
            }
        },
        stream=True,                  # Let requests yield the SSE lines
        timeout=None,                 # Large conversations ≠ premature timeout
    )

    # 3) Parse the event-stream line by line
    full_reply = ""
    for line in response.iter_lines():
        if not line:
            continue
        # Each non-empty line is a JSON object
        data = json.loads(line.decode("utf-8"))

        # Ollama sends a final line with {"done": true}
        if data.get("done"):
            break

        delta = data.get("message", {}).get("content", "")
        full_reply += delta
        yield full_reply               # incremental update to Gradio

In [3]:
demo = gr.ChatInterface(
    fn=stream_chat,
    chatbot=gr.Chatbot(label="Local LLM Chatbot (Ollama)"),
    additional_inputs=[
        gr.Dropdown(
            choices=AVAILABLE_MODELS,
            label="Select Model",
            value=AVAILABLE_MODELS[0],   # default = tinyllama
        )
    ],
    textbox=gr.Textbox(placeholder="Ask anything..."),
    title="🤖 Local Chatbot Assistant",
    theme="default",
)




C:\Users\deluc\AppData\Local\Temp\ipykernel_11096\1662960440.py:3: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(label="Local LLM Chatbot (Ollama)"),
c:\Users\deluc\Documents\Agentic-Code\projects\agents\.venv\Lib\site-packages\gradio\chat_interface.py:321: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


In [4]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
